In [43]:
import json
import yfinance as yf
import openai

from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

def retorna_catacao_acao_historica(
        ticker,
        periodo='1mo'
):
    ticker_obj = yf.Ticker(f'{ticker}.SA')
    hist = ticker_obj.history(period=periodo)['Close']
    hist.index = hist.index.strftime('%Y-%m-%d')
    hist = round(hist, 2)
    if len(hist) > 30:
        slice_size = int(len(hist) / 30)
        hist = hist.iloc[::-slice_size][::-1]
    return hist.to_json()

tools = [
    { 
        'type': 'function',
        'function': {
            'name': 'retorna_catacao_acao_historica',
            'description': 'retorna a cotação diária histórica para uma ação da bovespa',
            'parameters': {
                'type': 'object',
                'properties': {
                    'ticker': {
                        'type':'string',
                        'description': 'O ticker da ação. Exemplo: "ABEV3" para Ambev, "PETR4" para Petrobras, etc'
                    },
                    'periodo': {
                        'type':'string',
                        'description': 'Período de tempo (1d para 1 dia, 5d para 5 dias, 1mo para 1 mês, 3mo para 3 meses, 6mo para 6 meses, 1y para 1 ano, 2y para 2 anos, 5y para 5 anos, 10y para 10 anos)',
                        'enun': ['1d', '5d', '1mo', '6mo', '1y', '2y', '5y', 'ytd', 'max']
                    }
                }
            }
        }
    }    
]

funcoes_disponiveis = {'retorna_catacao_acao_historica' : retorna_catacao_acao_historica}

mensagens = [{'role': 'user', 'content': 'Qual a cotação da ambev agora?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo-0125',
    tools=tools,
    tool_choice='auto'
)

In [ ]:
tool_calls = resposta.choices[0].message.tool_calls

if tool_calls:
    mensagens.append(resposta.choices[0].message)
    for tool_call in tool_calls:
        func_name = tool_call.function.name
        function_to_call = funcoes_disponiveis[func_name]
        func_args = json.loads(tool_call.function.arguments)
        func_return = function_to_call(**func_args)
        mensagens.append({
            'tool_call_id': tool_call.id,
            'role': 'tool',
            'name': func_name,
            'content': func_return,
        })
    segunda_resposta = client.chat.completions.create(
        messages=mensagens,
        model='gpt-3.5-turbo-0125',
    )

    mensagens.append(segunda_resposta.choices[0].message)        